<a href="https://colab.research.google.com/github/Vasundhara-Boomi/Prediction-of-Neurodegenerative-disease-by-Hand-Drawing/blob/main/Prediction_By_Hand_Drawing_DL_ML_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'parkinsons-drawings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F178338%2F401677%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240501%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240501T083452Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D49a001df583cc397e372b01e50467fa5e3240aba35b111bce587e4a151ca4c2b015f6da7acbbf455c604f5be9f7b047f42f5a77b2e73ddc35538a88d28647624085c301383f83511fc24ad2acad8cc998634042d513fa62f66be889e7dd23bc485293c63fa6233f9cef7b2961b35f41e0a693d295efbc510befcdadc7fc2b60a35bb61891c1a7f93c0b116e0e13f423fa3e4b2c2df93922c8f6d8a2211010239be8fa8cd21f51f4706f93d216a68fdd5028cc9af7ef2dee5edc04a64f354c8929b59ff133d021baa2a88615219cd5ebbccc9d0949126238b6b2d27d2fba9c66a705b2e06e3b1cd95eeee0a9df092d6ffa563dd980e9fd178121a8de33ac21730'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 43444848 bytes downloaded
Downloaded and uncompressed: parkinsons-drawings
Data source import complete.


# HISTORY

#### Content
* Images of healthy and patients with Parkinsons drawing spirals and waves. The images are further divided into training and testing groups for comparing (or reproducing) the results of the original publication.

#### Acknowledgements
* The data came from the paper: Zham P, Kumar DK, Dabnichki P, Poosapadi Arjunan S and Raghav S (2017) Distinguishing Different Stages of Parkinson’s Disease Using Composite Index of Speed and Pen-Pressure of Sketching a Spiral. Front. Neurol. 8:435. doi: 10.3389/fneur.2017.00435

https://www.frontiersin.org/articles/10.3389/fneur.2017.00435/full

# PACKAGES AND LIBRARIES

In [2]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=UserWarning)

ModuleNotFoundError: No module named 'keras.utils.np_utils'

# PATH & LABELS

#### TRAIN-TEST PATH

In [ ]:
Spiral_Train_Path = Path("../input/parkinsons-drawings/spiral/training")
Spiral_Test_Path = Path("../input/parkinsons-drawings/spiral/testing")

In [ ]:
Spiral_Train_PNG_Path = list(Spiral_Train_Path.glob(r"*/*.png"))
Spiral_Test_PNG_Path = list(Spiral_Test_Path.glob(r"*/*.png"))

#### LABELS

In [ ]:
Spiral_Train_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Spiral_Train_PNG_Path))
Spiral_Test_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Spiral_Test_PNG_Path))

# TRANSFORMATION TO SERIES STRUCTURE

#### SPIRAL

In [ ]:
Spiral_Train_PNG_Path_Series = pd.Series(Spiral_Train_PNG_Path,name="PNG").astype(str)
Spiral_Train_PNG_Labels_Series = pd.Series(Spiral_Train_PNG_Labels,name="CATEGORY")

In [ ]:
Spiral_Test_PNG_Path_Series = pd.Series(Spiral_Test_PNG_Path,name="PNG").astype(str)
Spiral_Test_PNG_Labels_Series = pd.Series(Spiral_Test_PNG_Labels,name="CATEGORY")

# TRANSFORMATION TO DATAFRAME STRUCTURE

#### SPIRAL

In [ ]:
Main_Spiral_Train_Data = pd.concat([Spiral_Train_PNG_Path_Series,Spiral_Train_PNG_Labels_Series],axis=1)

In [ ]:
print(Main_Spiral_Train_Data.head(-1))

In [ ]:
Main_Spiral_Test_Data = pd.concat([Spiral_Test_PNG_Path_Series,Spiral_Test_PNG_Labels_Series],axis=1)

In [ ]:
print(Main_Spiral_Test_Data.head(-1))

# SHUFFLING

In [ ]:
Main_Spiral_Train_Data = Main_Spiral_Train_Data.sample(frac=1).reset_index(drop=True)
Main_Spiral_Test_Data = Main_Spiral_Test_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
print(Main_Spiral_Train_Data.head(-1))
print("---"*20)
print(Main_Spiral_Test_Data.head(-1))

# VISUALIZATION

In [ ]:
plt.style.use("dark_background")

In [ ]:
sns.countplot(Main_Spiral_Train_Data["CATEGORY"])
plt.show()

In [ ]:
sns.countplot(Main_Spiral_Test_Data["CATEGORY"])
plt.show()

In [ ]:
Main_Spiral_Train_Data['CATEGORY'].value_counts().plot.pie(figsize=(7,7))
plt.show()

In [ ]:
Main_Spiral_Test_Data['CATEGORY'].value_counts().plot.pie(figsize=(7,7))
plt.show()

In [ ]:
figure = plt.figure(figsize=(10,10))
x = plt.imread(Main_Spiral_Train_Data["PNG"][0])
plt.imshow(x)
plt.xlabel(x.shape)
plt.title(Main_Spiral_Train_Data["CATEGORY"][0])

In [ ]:
figure = plt.figure(figsize=(10,10))
x = plt.imread(Main_Spiral_Test_Data["PNG"][0])
plt.imshow(x)
plt.xlabel(x.shape)
plt.title(Main_Spiral_Test_Data["CATEGORY"][0])

In [ ]:
fig, axes = plt.subplots(nrows=5,
                        ncols=5,
                        figsize=(10,10),
                        subplot_kw={"xticks":[],"yticks":[]})

for i,ax in enumerate(axes.flat):
    ax.imshow(plt.imread(Main_Spiral_Train_Data["PNG"][i]))
    ax.set_title(Main_Spiral_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=5,
                        ncols=5,
                        figsize=(10,10),
                        subplot_kw={"xticks":[],"yticks":[]})

for i,ax in enumerate(axes.flat):
    ax.imshow(plt.imread(Main_Spiral_Test_Data["PNG"][i]))
    ax.set_title(Main_Spiral_Test_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

# CLASSIFIERS

#### TRANSFORMATION

In [ ]:
Spiral_New_JPG_Path = []
for i in range(0,72):
    x = cv2.imread(Main_Spiral_Train_Data["PNG"][i])
    x = np.array(x).astype("float32")
    x = x.mean()
    Spiral_New_JPG_Path.append(x)

In [ ]:
Spiral_New_JPG_Path_Series = pd.Series(Spiral_New_JPG_Path,name="PNG")

In [ ]:
print(Spiral_New_JPG_Path_Series)

In [ ]:
encode = LabelEncoder()

In [ ]:
Spiral_New_JPG_Labels = encode.fit_transform(Main_Spiral_Train_Data["CATEGORY"])

In [ ]:
Spiral_New_JPG_Labels_Series = pd.Series(Spiral_New_JPG_Labels,name="CATEGORY")

In [ ]:
print(Spiral_New_JPG_Labels_Series)

In [ ]:
Main_Spiral_New_Data = pd.concat([Spiral_New_JPG_Path_Series,Spiral_New_JPG_Labels_Series],axis=1)

In [ ]:
print(Main_Spiral_New_Data)

#### TRAIN & TEST

In [ ]:
x = Main_Spiral_New_Data[["PNG"]]
y = Main_Spiral_New_Data["CATEGORY"]

In [ ]:
print(x.shape)

In [ ]:
print(y.shape)

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.1,random_state=42)

In [ ]:
print(xTrain.shape)
print(xTest.shape)
print(yTrain.shape)
print(yTest.shape)

#### MODELS

In [ ]:
lj = LogisticRegression(solver="liblinear").fit(xTrain,yTrain)
gnb = GaussianNB().fit(xTrain,yTrain)
knnc = KNeighborsClassifier().fit(xTrain,yTrain)
cartc = DecisionTreeClassifier(random_state=42).fit(xTrain,yTrain)
rfc = RandomForestClassifier(random_state=42,verbose=False).fit(xTrain,yTrain)
gbmc = GradientBoostingClassifier(verbose=False).fit(xTrain,yTrain)
xgbc = XGBClassifier().fit(xTrain,yTrain)
lgbmc = LGBMClassifier().fit(xTrain,yTrain)
catbc = CatBoostClassifier(verbose=False).fit(xTrain,yTrain)

In [ ]:
modelsc = [lj,gnb,knnc,cartc,rfc,gbmc,xgbc,lgbmc,catbc]

In [ ]:
for model in modelsc:
    name = model.__class__.__name__
    predict = model.predict(xTest)
    R2CV = cross_val_score(model,xTest,yTest,verbose=False).mean()
    error = -cross_val_score(model,xTest,yTest,scoring="neg_mean_squared_error",verbose=False).mean()
    print(name + ": ")
    print("-" * 10)
    print("ACC-->",accuracy_score(yTest,predict))
    print("R2CV-->",R2CV*100)
    print("MEAN SQUARED ERROR-->",np.sqrt(error))
    print("-" * 30)

In [ ]:
lm = LinearRegression().fit(xTrain,yTrain)
pls = PLSRegression().fit(xTrain,yTrain)
ridge = Ridge().fit(xTrain,yTrain)
lasso = Lasso().fit(xTrain,yTrain)
elasticnet = ElasticNet().fit(xTrain,yTrain)
knnr = KNeighborsRegressor().fit(xTrain,yTrain)
cartr = DecisionTreeRegressor(random_state=42).fit(xTrain,yTrain)
baggr = BaggingRegressor(random_state=42,bootstrap_features=True,verbose=False).fit(xTrain,yTrain)
rfr = RandomForestRegressor(random_state=42,verbose=False).fit(xTrain,yTrain)
gbmr = GradientBoostingRegressor(verbose=False).fit(xTrain,yTrain)
xgbr = XGBRegressor().fit(xTrain,yTrain)
lgbmr = LGBMRegressor().fit(xTrain,yTrain)
catbr = CatBoostRegressor(verbose=False).fit(xTrain,yTrain)

In [ ]:
models = [lm,pls,ridge,lasso,elasticnet,knnr,
         cartr,baggr,rfr,gbmr,xgbr,lgbmr,catbr]

In [ ]:
for model in models:
    name = model.__class__.__name__
    R2CV = cross_val_score(model,xTest,yTest,scoring="r2").mean()
    error = -cross_val_score(model,xTest,yTest,scoring="neg_mean_squared_error").mean()
    print(name + ": ")
    print("-" * 10)
    print(R2CV)
    print(np.sqrt(error))
    print("-" * 30)

# IMAGE GENERATOR PROCESS

#### APPLYING GENERATOR

In [ ]:
Train_Generator = ImageDataGenerator(rescale=1./255,
                                    zoom_range=0.7,
                                    shear_range=0.7,
                                    rotation_range=50,
                                    horizontal_flip=True,
                                     brightness_range=[0.2,0.9],
                                     vertical_flip=True,
                                    validation_split=0.1)

In [ ]:
Train_Spiral_Set = Train_Generator.flow_from_dataframe(dataframe=Main_Spiral_Train_Data,
                                                   x_col="PNG",
                                                   y_col="CATEGORY",
                                                   color_mode="grayscale",
                                                   class_mode="categorical",
                                                       subset="training")

In [ ]:
Validation_Spiral_Set = Train_Generator.flow_from_dataframe(dataframe=Main_Spiral_Train_Data,
                                                   x_col="PNG",
                                                   y_col="CATEGORY",
                                                   color_mode="grayscale",
                                                   class_mode="categorical",
                                                       subset="validation")

In [ ]:
Test_Spiral_Set = Train_Generator.flow_from_dataframe(dataframe=Main_Spiral_Test_Data,
                                                   x_col="PNG",
                                                   y_col="CATEGORY",
                                                   color_mode="grayscale",
                                                   class_mode="categorical")

In [ ]:
print("TRAIN: ")
print(Train_Spiral_Set.class_indices)
print(Train_Spiral_Set.classes[0:5])
print(Train_Spiral_Set.image_shape)
print("---"*20)
print("VALIDATION: ")
print(Validation_Spiral_Set.class_indices)
print(Validation_Spiral_Set.classes[0:5])
print(Validation_Spiral_Set.image_shape)
print("---"*20)
print("TEST: ")
print(Test_Spiral_Set.class_indices)
print(Test_Spiral_Set.classes[0:5])
print(Test_Spiral_Set.image_shape)

#### TRANSFORMATION

In [ ]:
example_Image = Main_Spiral_Train_Data["PNG"][30]
Load_Image = image.load_img(example_Image,target_size=(210,210))
Array_Image = image.img_to_array(Load_Image)
Array_Image = Array_Image.reshape((1,) + Array_Image.shape)
i = 0
for batch in Train_Generator.flow(Array_Image,batch_size=5):
    plt.figure(i)
    Image = plt.imshow(image.img_to_array(batch[0]))
    i += 1
    if i % 4 == 0:
        break
plt.show()

# MODEL

#### CNN

In [ ]:
Call_Back_Early = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                   patience=7,
                                                   mode="max")
Call_Back_Check = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                     save_best_only=True,
                                                     filepath="./my_model")

In [ ]:
Model_One = Sequential()

#
Model_One.add(Conv2D(2,(15,15),activation="relu",
                 input_shape=(256,256,1)))
Model_One.add(MaxPooling2D((2,2)))
Model_One.add(Dropout(0.2))
#
Model_One.add(Conv2D(4,(10,10),activation="relu",
                 strides=(2,2)))
Model_One.add(MaxPooling2D((2,2)))
Model_One.add(Dropout(0.2))

#
Model_One.add(Flatten())
Model_One.add(Dropout(0.5))
Model_One.add(Dense(512,activation="relu"))
Model_One.add(Dense(2,activation="softmax"))

In [ ]:
Model_One.compile(optimizer=RMSprop(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
CNN_Model_One = Model_One.fit(Train_Spiral_Set,
                      validation_data=Validation_Spiral_Set,
                            callbacks=[Call_Back_Check,Call_Back_Early],
                      epochs=50)

In [ ]:
Model_Results = Model_One.evaluate(Test_Spiral_Set)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

#### ANN

In [ ]:
Model_Two = tf.keras.models.Sequential([
  # inputs
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Flatten(input_shape=(113,)),
  # hiddens layers
    tf.keras.layers.Dense(10, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  # output layer
  tf.keras.layers.Dense(2,activation="softmax")
])

In [ ]:
lossfunc = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [ ]:
Model_Two.compile(optimizer='adam', loss=lossfunc, metrics=['accuracy'])

In [ ]:
ANN_Model = Model_Two.fit(Train_Spiral_Set,
                      validation_data=Validation_Spiral_Set,
                      epochs=50,batch_size=10)

In [ ]:
Model_Results_Two = Model_Two.evaluate(Test_Spiral_Set)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

# PREDICTION PROCESS

#### CNN

In [ ]:
Prediction_One = Model_One.predict(Test_Spiral_Set)
Prediction_One = Prediction_One.argmax(axis=-1)

In [ ]:
Predict_Proba_One = Model_One.predict_proba(Test_Spiral_Set)

#### ANN

In [ ]:
Prediction_Two = Model_Two.predict(Test_Spiral_Set)
Prediction_Two = Prediction_Two.argmax(axis=-1)

In [ ]:
Predict_Proba_Two = Model_Two.predict_proba(Test_Spiral_Set)

#### COMMUNITY

In [ ]:
Main_Predict = 0.5 * (Predict_Proba_One + Predict_Proba_Two)

In [ ]:
print(Main_Predict)

In [ ]:
fig, axes = plt.subplots(nrows=5,
                         ncols=5,
                         figsize=(20, 20),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(Main_Spiral_Test_Data["PNG"].iloc[i]))
    ax.set_title(f"PREDICTION:{Main_Predict[i]}")
plt.tight_layout()
plt.show()